# Process fgmax grid results and plot

To process fgmax results after doing a run.

In [ ]:
%matplotlib inline

In [ ]:
from pylab import *

In [ ]:
import os,sys
import glob
from importlib import reload
from clawpack.geoclaw import topotools, dtopotools
from clawpack.visclaw import colormaps
from scipy.interpolate import RegularGridInterpolator
import matplotlib as mpl
from matplotlib import colors

In [ ]:
sys.path.insert(0,'../../new_python')
import region_tools, plottools
import fgmax_tools, fgmax_routines, kmltools

## Set some things...

Specify the directory to read results from, and some other settings:

In [ ]:
save_figs = False              # make png files for figures?
make_new_fgmax_nc_file = True  # make new netcdf file of fgmax results (possibly overwrites)?

rundir = os.path.abspath('.')
outdir = os.path.join(rundir, '_output') 

print('Will read fgmax results from outdir = \n  ', outdir)

In [ ]:
use_force_dry = True
if use_force_dry:
    fname_force_dry = os.path.join(rundir, 'input_files', 'force_dry_init.data')
    
adjust_by_dz = True
if adjust_by_dz:
    dtopo_path = os.path.join(rundir, 'input_files', 'seattlefault_uniform.tt3')

In [ ]:
def savefigp(fname):
    global save_figs
    if save_figs:
        savefig(fname)
        print('Created ', fname)
    else:
        print('save_figs = False')

## Read in and process the fgmax results from the latest run


In [ ]:
t_files = glob.glob(outdir + '/fort.t0*')
times = []
for f in t_files:
    lines = open(f,'r').readlines()
    for line in lines:
        if 'time' in line: 
            t = float(line.split()[0])
    times.append(t)
times.sort()
print('Output times found: ',times)
if len(times) > 0:
    t_hours = times[-1] / 3600.
    print('\nfgmax results are presumably from final time: %.1f seconds = %.2f hours'\
          % (times[-1], t_hours))
else:
    t_hours = nan

In [ ]:
# Read fgmax data:
fg = fgmax_tools.FGmaxGrid()
fgmax_input_file_name = outdir + '/fgmax_header.data'
print('fgmax input file: \n  %s' % fgmax_input_file_name)
fg.read_input_data(fgmax_input_file_name)

fg.read_output(outdir=outdir)
xx = fg.X
yy = fg.Y

In [ ]:
# convert to masked array on uniform grid for .nc file and plots:

fgm = fgmax_tools.FGmaxMaskedGrid()
dx = dy = 1./(3*3600.)  # For 1/3 arcsecond fgmax grid

# convert to arrays and create fgm.X etc.
fgm.convert_lists_to_arrays(fg,dx,dy) 

# 1d versions of X and Y arrays:
fgm.x = fgm.X[0,:]
fgm.y = fgm.Y[:,0]

In [ ]:
# compute subsidence/uplift at each fgmax point:

if adjust_by_dz:
    dtopo = dtopotools.DTopography()
    dtopo.read(dtopo_path, dtopo_type=3)
    x1d = dtopo.X[0,:]
    y1d = dtopo.Y[:,0]
    dtopo_func = RegularGridInterpolator((x1d,y1d), dtopo.dZ[-1,:,:].T, 
                    method='linear', bounds_error=False, fill_value=0.)
    dz = dtopo_func(list(zip(ravel(fgm.X), ravel(fgm.Y))))
    fgm.dz = reshape(dz, fgm.X.shape)
    print('Over fgmax extent, min(dz) = %.2f m, max(dz) = %.2f m' \
         % (dz.min(), dz.max()))
else:
    fgm.dz = zeros(fgm.X.shape)

fgm.B0 = fgm.B - fgm.dz   # original topo before subsidence/uplift

In [ ]:
if use_force_dry:
    print('Reading force_dry from ',fname_force_dry)
    force_dry = topotools.Topography()
    force_dry.read(fname_force_dry, topo_type=3)
    i1 = int(round((fgm.x[0]-force_dry.x[0])/dx))
    i2 = int(round((fgm.x[-1]-force_dry.x[0])/dx))
    j1 = int(round((fgm.y[0]-force_dry.y[0])/dy))
    j2 = int(round((fgm.y[-1]-force_dry.y[0])/dy))
    if (i1<0) or (i2-i1+1 != len(fgm.x)) or \
       (j1<0) or (j2-j1+1 != len(fgm.y)):
        print('*** force_dry does not cover fgm extent, not using')
        use_force_dry = False
        fgm.force_dry_init = None
    else:
        fgm.force_dry_init = force_dry.Z[j1:j2+1, i1:i2+1]
else:
    fgm.force_dry_init = None
    print('*** use_force_dry is False')

if fgm.force_dry_init is not None:
    fgm.h_onshore = ma.masked_where(fgm.force_dry_init==0, fgm.h)
else:
    fgm.h_onshore = ma.masked_where(fgm.B0 < 0., fgm.h)

In [ ]:
print('Number of fgmax points: ', fgm.h.count())

In [ ]:
zmin = -60.
zmax = 40.
land_cmap = colormaps.make_colormap({ 0.0:[0.1,0.4,0.0],
                                     0.25:[0.0,1.0,0.0],
                                      0.5:[0.8,1.0,0.5],
                                      1.0:[0.8,0.5,0.2]})

sea_cmap = colormaps.make_colormap({ 0.0:[0,0,1], 1.:[.8,.8,1]})

cmap, norm = colormaps.add_colormaps((land_cmap, sea_cmap),
                                     data_limits=(zmin,zmax),
                                     data_break=0.)                                   

def plotZ(Z, show_cb=True):
    pc = plottools.pcolorcells(fgm.X, fgm.Y, Z, cmap=cmap, norm=norm)  
    if show_cb:
        cb = colorbar(pc,shrink=0.5)
        cb.set_label('meters')
    #axis([-122.76,-122.525,47.95,48.2])
    gca().set_aspect(1./cos(48*pi/180.))
    ticklabel_format(useOffset=False)
    xticks(rotation=20);
    
figure(figsize=(10,6))
subplot(121)
plotZ(fgm.B, show_cb=False)
title('GeoClaw B');

if fgm.force_dry_init is not None:
    print('Found force_dry_init array')
    subplot(122)
    mask_all_but_dryneg = logical_or(logical_or(fgm.B.mask, 
                                                logical_not(fgm.force_dry_init)), 
                                     fgm.B0>0)
    B_dryneg = ma.masked_array(fgm.B.data, mask=mask_all_but_dryneg)
    plotZ(fgm.B, show_cb=False)
    
    sea_cmap_dry = colormaps.make_colormap({ 0.0:[1.0,0.6,0.6], 1.:[1.0,0.6,0.6]})
    cmap_dry, norm_dry = colormaps.add_colormaps((land_cmap, sea_cmap_dry),
                                         data_limits=(zmin,zmax),
                                         data_break=0.)
    B0_dryneg = ma.masked_array(fgm.B0.data, mask=mask_all_but_dryneg)
    plottools.pcolorcells(fgm.X, fgm.Y, B0_dryneg, cmap=cmap_dry, norm=norm_dry)
    title('B0, with dry regions below MHW pink')
    savefigp('geoclaw_topo_and_dry.png')
else:
    print('No force_dry_init array')

In the plot above, "GeoClaw B" refers to the cell-averaged topography value used by GeoClaw and stored with the fgmax output, and is generally recorded after any subsidence/uplift.  The colors are blues for values of $B < 0$ and greens/brown for $B > 0$.  If there's a plot on the right, it shows as pink any areas that were initialized as dry in spite of having $B_0 < 0$, where $B_0$ is the initial topography ($B$ corrected by $dz$).

## Plot maximum flow depth

In [ ]:
bounds_depth = array([1e-6,0.25,0.5,0.75,1,1.25,1.5])
#bounds_depth = array([1e-6,0.5,1.0,1.5,2,2.5,3.0])


cmap_depth = colors.ListedColormap([[.7,.7,1],[.5,.5,1],[0,0,1],\
                 [1,.7,.7], [1,.4,.4], [1,0,0]])

# Set color for value exceeding top of range to purple:
cmap_depth.set_over(color=[1,0,1])

# Set color for land points without inundation to light green:
cmap_depth.set_under(color=[.7,1,.7])

norm_depth = colors.BoundaryNorm(bounds_depth, cmap_depth.N)
    

figure(figsize=(8,8))
pc = plottools.pcolorcells(fgm.X, fgm.Y, fgm.h_onshore, cmap=cmap_depth, norm=norm_depth)
cb = colorbar(pc, extend='max', shrink=0.7)
cb.set_label('meters')
contour(fgm.X, fgm.Y, fgm.B0, [0], colors='g')

gca().set_aspect(1./cos(48*pi/180.))
ticklabel_format(useOffset=False)
xticks(rotation=20)
title('Maximum flow depth over %.2f hours' % t_hours)
savefigp('h_onshore.png')

In the plot above, green shows fgmax points that never got wet.  The green contour shows `B0 = 0`, and note that some of the initially dry region below MHW never got wet (over the limited duration of this simulation).

White areas are masked out either because they were not fgmax points or because they were initially wet. 

Regions colored blue or red are initially dry fgmax points that did get wet during the tsunami, with color showing the maximum depth of water recorded.

## Plot maximum speed

In [ ]:
bounds_speed = np.array([1e-6,0.5,1.0,1.5,2,2.5,3,4.5,6])
cmap_speed = mpl.colors.ListedColormap([[.9,.9,1],[.6,.6,1],\
                 [.3,.3,1],[0,0,1], [1,.8,.8],\
                 [1,.6,.6], [1,.3,.3], [1,0,0]])

# Set color for value exceeding top of range to purple:
cmap_speed.set_over(color=[1,0,1])

# Set color for land points without inundation to light green:
cmap_speed.set_under(color=[.7,1,.7])

norm_speed = colors.BoundaryNorm(bounds_speed, cmap_speed.N)

figure(figsize=(8,8))
pc = plottools.pcolorcells(fgm.X, fgm.Y, fgm.s, cmap=cmap_speed, norm=norm_speed)
cb = colorbar(pc, extend='max', shrink=0.7)
cb.set_label('m/s')
contour(fgm.X, fgm.Y, fgm.B0, [0], colors='g')

gca().set_aspect(1./cos(48*pi/180.))
ticklabel_format(useOffset=False)
xticks(rotation=20)
title('Maximum speed over %.2f hours' % t_hours)
savefigp('speed.png')

The plot above shows the maximum speed at each fgmax point. The points colored green remained dry over this simulation. The green contour shows `B0 = 0`.

White areas are masked out because they were not fgmax points. Regions colored blue or red are either offshore (initially wet) or onshore points that got wet, colored by the maximum water speed $s = \sqrt{u^2 + v^2}$ over the simulation. 

## Plots for Google Earth overlays

Tne new version of `kmltools` includes some tools to make png files that display properly on Google Earth.  The png files have no axes and have the dimension and dpi set properly so that there is an integer number of pixels in each grid cell so cell edges are sharp when zooming in.

We make three png files and then make a kml file that can be used to open all three.

In [ ]:
kml_dir = 'fgmax_results_kmlfiles'
os.system('mkdir -p %s' % kml_dir)
print('Will put png and kml files in %s' % kml_dir)

In [ ]:
h_wet_onshore = ma.masked_where(fgm.h_onshore==0., fgm.h_onshore)
png_filename=kml_dir+'/h_onshore_max_for_kml.png'
fig,ax,png_extent,kml_dpi = kmltools.pcolorcells_for_kml(fgm.x, fgm.y, h_wet_onshore,
                                                 png_filename=png_filename,
                                                 dpc=2, cmap=cmap_depth, norm=norm_depth)

In [ ]:
speed = ma.masked_where(fgm.h==0., fgm.s)
png_filename = '%s/speed_max_for_kml.png' % kml_dir
fig,ax,png_extent,kml_dpi = kmltools.pcolorcells_for_kml(fgm.x, fgm.y, speed, 
                                                 png_filename=png_filename,
                                                 dpc=2, cmap=cmap_speed, norm=norm_speed)

In [ ]:
stays_dry = ma.masked_where(fgm.h>0., fgm.h)
png_filename = '%s/stays_dry_for_kml.png' % kml_dir
fig,ax,png_extent,kml_dpi = kmltools.pcolorcells_for_kml(fgm.x, fgm.y, stays_dry, 
                                                 png_filename=png_filename,
                                                 dpc=2, cmap=cmap_speed, norm=norm_speed)

### Make the kml file to display these three png files

Then you can open `fgmax_results_kmlfiles/fgmax_results.kml` in Google Earth to view them.

In [ ]:
png_files=['h_onshore_max_for_kml.png', 'speed_max_for_kml.png','stays_dry_for_kml.png']
png_names=['max depth onshore','max speed','stays dry']

kmltools.png2kml(png_extent, png_files=png_files, png_names=png_names, 
                 name='fgmax_results',
                 fname='%s/fgmax_results.kml' % kml_dir,
                 radio_style=False)

print('Contents of %s:' % kml_dir)
for f in glob.glob('%s/*' % kml_dir):
    print('    ',f)